In [1]:
!pip install vfb-connect --upgrade

Requirement already up-to-date: vfb-connect in /root/venv/lib/python3.7/site-packages (1.2.4)
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.



### VFB connect API overview

The VFB connect API provides programmatic access to the databases underlying [Virtual Fly Brain](http://virtualflybrain.org). 

At the core of Virtual Fly Brain is a set of curated terms for Drosophila neuro-anatomy organised into a queryable classification, including terms for brain regions, e.g. [nodulus](http://virtualflybrain.org/reports/FBbt_00003680) and neurons e.g. [MBON01](http://virtualflybrain.org/reports/FBbt_00100234). These terms are used to annotate and classify individual brain regions and neurons in images and connectomics data. For example the term MBON01 is used to classify individual [neurons from sources including the CATMAID-FAFB and Neuprint-HemiBrain databases](https://v2.virtualflybrain.org/org.geppetto.frontend/geppetto?q=FBbt_00100234,ListAllAvailableImages). VFB stores both [registered 3D images](http://virtualflybrain.org/reports/VFB_00101382) and [connectomics data](https://v2-dev.virtualflybrain.org/org.geppetto.frontend/geppetto?q=VFB_00101382,ref_neuron_neuron_connectivity_query) (where available) for all of these neurons.

A single VfbConnect object wraps connections and canned queries against all open VFB databases. It includes methods for retreiving metadata about anatomy, individual brain regions and neurons including IDs for these that can be used for queries against other databases (e.g. CATMAID & neuprint).  It provides methods for downloading images and connectomics data. It provides access to sophisticated queries for anatomical classes and individual neurons accoring to their classification & properties. 

In [2]:
# Import lib and initialise object
from vfb_connect.cross_server_tools import VfbConnect
vc = VfbConnect()

### Locations for methods under a `VfbConnect` object.

1. Under `vc.neo_query_wrapper` are a set of methods that take lists of IDs as a primary argument and return metadata.  Details of available Methods can be found [here] #Link TBA
2. Directly under `vc` are:
   - a set of methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes.
   - methods for querying connectivity and similarity
3. Direct access to API queries is provided under the 'nc' and 'oc' attributes for Neo4J and OWL queries respectively. We will not cover details of how to use these here.

### 1. `vc.neo_query_wrapper` methods overview

`vc.neo_query_wrapper` TermInfo queries return the results of a VFB Term Information window as JSON, following the [VFB_JSON standard](), or a summary that can easily be converted into a DataFrame.

In [3]:
# A query for full TermInfo.  This probably produces more information than you will need for most purposes.

vc.neo_query_wrapper.get_type_TermInfo(['FBbt_00003686'])

[{'term': {'core': {'iri': 'http://purl.obolibrary.org/obo/FBbt_00003686',
    'symbol': '',
    'types': ['Entity',
     'Anatomy',
     'Nervous_system',
     'Cell',
     'Neuron',
     'Class'],
    'label': 'Kenyon cell',
    'short_form': 'FBbt_00003686'},
   'description': ['Intrinsic neuron of the mushroom body. They have tightly-packed cell bodies, situated in the rind above the calyx of the mushroom body (Ito et al., 1997). Four short fascicles, one per lineage, extend from the cell bodies of the Kenyon cells into the calyx (Ito et al., 1997). These 4 smaller fascicles converge in the calyx where they arborize and form pre- and post-synaptic terminals (Christiansen et al., 2011), with different Kenyon cells receiving input in different calyx regions/accessory calyces (Tanaka et al., 2008). They emerge from the calyx as a thick axon bundle referred to as the peduncle that bifurcates to innervate the dorsal and medial lobes of the mushroom body (Tanaka et al., 2008).'],
   'com

In [4]:
# A query for summary info
import pandas as pd

summary = vc.neo_query_wrapper.get_type_TermInfo(['FBbt_00003686'], summary=True)
summary_tab = pd.DataFrame.from_records(summary)
summary_tab

,label,symbol,id,tags,parents_label,parents_id
0,Kenyon cell,,FBbt_00003686,Entity|Anatomy|Nervous_system|Cell|Neuron|Class,supraesophageal ganglion neuron|mushroom body ...,FBbt_00001366|FBbt_00007484


In [5]:
# A different method is needed to info about individual neurons

summary = vc.neo_query_wrapper.get_anatomical_individual_TermInfo(['VFB_jrchjrch'], summary=True)
summary_tab = pd.DataFrame.from_records(summary)
summary_tab


,label,symbol,id,tags,parents_label,parents_id,data_source,accession,templates,dataset,license
0,5-HTPLP01_R - 1324365879,,VFB_jrchjrch,Entity|has_image|Adult|Anatomy|has_neuron_conn...,adult serotonergic PLP neuron,FBbt_00110945,neuprint_JRC_Hemibrain_1point1,1324365879,JRC_FlyEM_Hemibrain|JRC2018Unisex,Xu2020NeuronsV1point1,https://creativecommons.org/licenses/by/4.0/le...


The `neo_query_wrapper` also includes methods for mapping between IDs from different sources. 

In [6]:
# Some bodyIDs of HemiBrain neurons from the neuprint DataBase:
bodyIDs = [1068958652, 571424748, 1141631198]
vc.neo_query_wrapper.xref_2_vfb_id(map(str, bodyIDs)) # Note IDs must be strings

{'1068958652': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwda'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjwda'}],
 '571424748': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwct'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjwct'}],
 '1141631198': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjw8r'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuprint_JRC_Hemibrain_1point1', 'vfb_id': 'VFB_jrchjw8r'}]}

In [7]:
# xref queries can be constrained by DB. Results can optionally be reversed

vc.neo_query_wrapper.xref_2_vfb_id(map(str, bodyIDs), db = 'neuprint_JRC_Hemibrain_1point1' , reverse_return=True)

{'VFB_jrchjw8r': [{'acc': '1141631198',
   'db': 'neuprint_JRC_Hemibrain_1point1'}],
 'VFB_jrchjwct': [{'acc': '571424748',
   'db': 'neuprint_JRC_Hemibrain_1point1'}],
 'VFB_jrchjwda': [{'acc': '1068958652',
   'db': 'neuprint_JRC_Hemibrain_1point1'}]}

### `vc` direct methods overview

2a Methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes or individual neurons

In [8]:
KC_types = vc.get_subclasses("'Kenyon cell'", summary=True)
pd.DataFrame.from_records(KC_types)

Running query: FBbt:00003686
Query URL: http://owl.virtualflybrain.org/kbs/vfb/subclasses?object=FBbt%3A00003686&prefixes=%7B%22FBbt%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FFBbt_%22%2C+%22RO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FRO_%22%2C+%22BFO%22%3A+%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FBFO_%22%7D&direct=False
Query results: 37


,label,symbol,id,tags,parents_label,parents_id
0,adult alpha'/beta' Kenyon cell,,FBbt_00049834,Entity|Adult|Anatomy|Nervous_system|Cell|Neuro...,alpha'/beta' Kenyon cell|adult Kenyon cell,FBbt_00100249|FBbt_00049825
1,immature Kenyon cell,,FBbt_00047995,Entity|Anatomy|Nervous_system|Cell|Neuron|Class,Kenyon cell,FBbt_00003686
2,gamma Kenyon cell,,FBbt_00100247,Entity|Anatomy|Nervous_system|Cell|Neuron|Class,Kenyon cell,FBbt_00003686
3,adult Kenyon cell,,FBbt_00049825,Entity|Adult|Anatomy|Nervous_system|Cell|Neuro...,adult MBp lineage neuron|Kenyon cell,FBbt_00110577|FBbt_00003686
4,gamma main Kenyon cell,KCg-m,FBbt_00111061,Entity|Adult|Anatomy|Nervous_system|Cell|Neuro...,Kenyon cell of main calyx|adult gamma Kenyon cell,FBbt_00047926|FBbt_00049828
5,alpha'/beta' anterior-posterior type 1 Kenyon ...,,FBbt_00049859,Entity|Adult|Anatomy|Nervous_system|Cell|Neuro...,alpha'/beta' anterior-posterior type 1 Kenyon ...,FBbt_00049836
6,alpha/beta Kenyon cell,,FBbt_00100248,Entity|Neuron|Adult|Anatomy|Nervous_system|Cel...,cholinergic neuron|adult Kenyon cell,FBbt_00007173|FBbt_00049825
7,gamma-s4 Kenyon cell,KCg-s4,FBbt_00049832,Entity|Adult|Anatomy|Nervous_system|Cell|Neuro...,gamma-s Kenyon cell,FBbt_00049830
8,two-claw Kenyon cell,,FBbt_00047997,Entity|Anatomy|Nervous_system|Cell|Neuron|Class,multi-claw Kenyon cell,FBbt_00047994
9,single-claw Kenyon cell,,FBbt_00047993,Entity|Anatomy|Nervous_system|Cell|Neuron|Class,Kenyon cell,FBbt_00003686


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dcbbcbb0-31bf-4ff6-86ec-96d450512668' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>